# Scalar Field

We first introduce some basic ideas and operations

In [1]:
import taichi as ti
import taichi.math as tm

ti.init(arch=ti.gpu)

# Declares a 0D scalar field whose data type is f32
f_0d = ti.field(ti.f32, shape=())  # 0D field: a single scalar
print("f_0d: ",f_0d, f_0d.shape)

f_1d = ti.field(ti.i32, shape=9)  # A 1D field of length 9:  a 1D array of scalars

print("f_1d: ",f_1d, f_1d.shape)

# Use None to access a 0D field and use 0 to access the first element of a 1D field
print("Indexing: ",f_0d[None], f_1d[0]) 


f_2d = ti.field(int, shape=(3, 6))
print("f_2d: ",f_2d, f_2d.shape)
# Taichi only supports fields of dimensions ≤ 8! But I think it's fine for the project

# We could assign values to fields:
f_0d[None] = 10.0
print("f_0d value assigned: ",f_0d)

# We could also use iteration to assign values to a nD field
@ti.kernel
def loop_over_1d():
  for i in range(9):
      f_1d[i] = i

loop_over_1d()
print("f_1d value assigned: ",f_1d)

#Similarly to higher dimensional field
@ti.kernel
def loop_over_2d():
  for i, j in f_2d:
      f_2d[i, j] = i

loop_over_2d()
print("f_2d value assigned: ",f_2d)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
[Taichi] version 1.7.2, llvm 15.0.5, commit 0131dce9, osx, python 3.9.21
[Taichi] Starting on arch=metal


[I 03/06/25 14:01:36.726 684364] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


f_0d:  0.0 ()
f_1d:  [0 0 0 0 0 0 0 0 0] (9,)
Indexing:  0.0 0
f_2d:  [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]] (3, 6)
f_0d value assigned:  10.0
f_1d value assigned:  [0 1 2 3 4 5 6 7 8]
f_2d value assigned:  [[0 0 0 0 0 0]
 [1 1 1 1 1 1]
 [2 2 2 2 2 2]]


We could use field to create an image as example below

In [2]:
width, height = 128,128
# Creates a 128x128 scalar field, each of its elements representing a pixel value (f32)
gray_scale_image = ti.field(dtype=ti.f32, shape=(width, height))
print(gray_scale_image.shape)

@ti.kernel
def fill_image():
    # Fills the image with random gray
    for i,j in gray_scale_image:
        gray_scale_image[i,j] = ti.random()

fill_image()

GUI = False
# Creates a GUI of the size of the gray-scale image
if GUI == True:
    gui = ti.GUI('gray-scale image of random values', (width, height))
    while gui.running:
        gui.set_image(gray_scale_image)
        #fill_image()
        gui.show()

# However, it's important to keep in mind that slicing operations are not defined(allowed in Taichi)
# For example, for x in f_2d[0]:... ; f_2d[0][3:] = [4, 5, 6] will fail!

(128, 128)


In [3]:
# Or there's another simple method to fill the field
print(f_2d)
f_2d.fill(-1)
print(f_2d)

[[0 0 0 0 0 0]
 [1 1 1 1 1 1]
 [2 2 2 2 2 2]]
[[-1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1]]


# Vector Field

We could consider ti.field as a scalar fields, where the shape of the field describes the n-Dimensional space, and for every position in the space there's a scalar value.

On the otherhadn, ti.Vector.field is a different thing. Basically it means at each position in the space there's a vector.

As you see, Taichi and torch consider things in quite different ways.

In [ ]:
# Declares a 3x3 vector field (spatial position) comprising 2D vectors (vector value)
f = ti.Vector.field(n=2, dtype=ti.f32, shape=(3, 3))
print("vector field: ",f,f.shape)


vector field:  [[[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]] (3, 3)


In [5]:
import time

#ti.init(arch=ti.cpu)
ti.init(arch=ti.gpu)
# This is a more practical 
n, w, h = 3, 128, 64 #n is the dimension of the vector in the space
vec_field_static = ti.Vector.field(n, dtype=float, shape=(w, h))
vec_field_dynamic = ti.Vector.field(n, dtype=float, shape=(w, h))


@ti.kernel
def fill_vector_static():
    for i, j in vec_field_static:
        for k in ti.static(range(n)):  # use ti.static, which unrolls the inner loops
            vec_field_static[i, j][k] = ti.random()


@ti.kernel
def fill_vector_dynamic():
    for i, j in vec_field_dynamic:
        for k in range(n):  # not use ti.static
            vec_field_dynamic[i, j][k] = ti.random()  # this is how you fill in the value for vectors


def benchmark(func, name, repeat=100):
    start_time = time.time()
    for _ in range(repeat):
        func()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"{name} ,time: {elapsed_time:.6f} seconds")


# Compare time
benchmark(fill_vector_static, "using ti.static")
benchmark(fill_vector_dynamic, "not using ti.static")

[Taichi] Starting on arch=metal
using ti.static ,time: 0.007735 seconds
not using ti.static ,time: 0.004521 seconds


In [6]:
print(vec_field_static[0,1][0]) # specify the position and the indexing is ok
try:
    vec_field_static[0,1,0] # direct slicing the field is not ok
except:
    print("NOT OK")

0.34879717230796814
NOT OK


# Matrix Field

Similar, shape controls the position of vectors, except this time the vector is not $\mathbb{R}^n$, rather it's $\mathbb{R}^{n \times m}$

In [7]:
tensor_field = ti.Matrix.field(n=3, m=2, dtype=ti.f32, shape=(300, 400, 500))
print(tensor_field.shape)
print(tensor_field[0,0,0])

#Similar, shape controls the position of vectors, except this time the vector is in

(300, 400, 500)
[[0. 0.]
 [0. 0.]
 [0. 0.]]


# AutoDiff

## Tape for Scalar Value Function

We first give a hardcoded differentiation as an example to show how we keep track of gradient information in general

In [8]:

# Hardcoded Differentiation

ti.init(arch=ti.gpu)

x = ti.field(dtype=ti.f32, shape=(), needs_grad=True)
y = ti.field(dtype=ti.f32, shape=(), needs_grad=True)

@ti.kernel
def compute_y():
    y[None] = ti.sin(x[None])

with ti.ad.Tape(y):
    compute_y()  # The tape will automatically compute dy/dx and save the results in x.grad

print('dy/dx =', x.grad[None], ' at x =', x[None])

print("Note x is a scalar field here",type(x.grad))


[Taichi] Starting on arch=metal
dy/dx = 1.0  at x = 0.0
Note x is a scalar field here <class 'taichi.lang.field.ScalarField'>


In [9]:
import taichi as ti

N = 8
dt = 1e-5

x = ti.Vector.field(2, dtype=ti.f32, shape=N, needs_grad=True)  # particle positions
v = ti.Vector.field(2, dtype=ti.f32, shape=N)  # particle velocities
U = ti.field(dtype=ti.f32, shape=(), needs_grad=True)  # potential energy


@ti.kernel
def compute_U():
    for i, j in ti.ndrange(N, N):
        r = x[i] - x[j] 
        # r.norm(1e-3) is equivalent to ti.sqrt(r.norm()**2 + 1e-3)
        # This is to prevent 1/0 error which can cause wrong derivative
        U[None] += -1 / r.norm(1e-3)  # U += -1 / |r|


@ti.kernel
def advance():
    for i in x:
        v[i] += dt * -x.grad[i]  # dv/dt = -dU/dx
    for i in x:
        x[i] += dt * v[i]  # dx/dt = v


def substep():
    #ti.ad.Tape() can only track a 0D field as the output variable!!!!!!!!!!
    with ti.ad.Tape(loss=U):
        # Kernel invocations in this scope will later contribute to partial derivatives of
        # U with respect to input variables such as x.
        compute_U()  # The tape will automatically compute dU/dx and save the results in x.grad
    advance()


@ti.kernel
def init():
    for i in x:
        x[i] = [ti.random(), ti.random()]


init()
use_gui = False
if use_gui == True:
    gui = ti.GUI('Autodiff gravity')
    while gui.running:
        for i in range(50):
            substep()
        gui.circles(x.to_numpy(), radius=3)
        gui.show()

In [10]:
x = ti.Vector.field(2, dtype=ti.f32, shape=N, needs_grad=True)  # particle positions
v = ti.Vector.field(2, dtype=ti.f32, shape=N)  # particle velocities
U = ti.field(dtype=ti.f32, shape=(), needs_grad=True)  # potential energy

print(x, type(x), x.shape) #I guess the 2 refers to the dimension
print(U[None], U.shape) #indexing for 0D object

print(x.shape,x[N+1]) #Taichi doesn't have index out of range issue (OFR value is just some meaningless value, but no error is reported)


[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]] <class 'taichi.lang.matrix.MatrixField'> (8,)
0.0 ()
(8,) [0. 0.]


Not use GUI, let's simply see how the gradient updates, look at numbers.

Here we provide some understanding

For particle $ i $, the potential energy function is given by:

$$
U_i = \sum_{j} -\frac{1}{|x_i - x_j|}
$$

To compute the gradient $ \frac{dU}{dx_i} $, we differentiate $ U_i $:

$$
\frac{dU}{dx_i} = \sum_{j} \frac{(x_i - x_j)}{|x_i - x_j|^3}
$$

This gradient points in the direction of the gravitational force acting on the particle.

In Taichi, `x.grad[i]` stores $ \frac{dU}{dx_i} $, which corresponds to the force experienced by particle $ i $, following the relationship:

$$
F_i = -\nabla U = -\frac{dU}{dx_i}
$$

Thus, the values in `x.grad[i]` represent the net force acting on particle $ i $.


In [11]:
import taichi as ti

ti.init()

N = 8
dt = 1e-3  
x = ti.Vector.field(2, dtype=ti.f32, shape=N, needs_grad=True)  # particle positions
v = ti.Vector.field(2, dtype=ti.f32, shape=N)  # particle velocities
U = ti.field(dtype=ti.f32, shape=(), needs_grad=True)  # potential energy


@ti.kernel
def compute_U():
    for i, j in ti.ndrange(N, N):
        r = x[i] - x[j] 
        # r.norm(1e-3) is equivalent to ti.sqrt(r.norm()**2 + 1e-3)
        # This is to prevent 1/0 error which can cause wrong derivative
        U[None] += -1 / r.norm(1e-3)  # U += -1 / |r|


@ti.kernel
def advance():
    for i in x:
        v[i] += dt * -x.grad[i]  # dv/dt = -dU/dx
    for i in x:
        x[i] += dt * v[i]  # dx/dt = v


def substep():
    with ti.ad.Tape(loss=U):
        compute_U()
    print("See gradient: \n",x.grad)
    advance()


@ti.kernel
def init():
    for i in x:
        x[i] = [ti.random() * 2 - 1, ti.random() * 2 - 1] 

init()

print("First and out of range vectors:\n",x[0],x[N+1])
for times in range(20):
    for i in range(50):
        substep()
        print("x[0] =", x[0])
        #print("x:", x.to_numpy()) 
        print("x[N+1] =", x[N+1])
    #print("x[0] =", x[0])
    #print("x:", x.to_numpy()) 
    #print("x[N+1] =", x[N+1])


[Taichi] Starting on arch=x64
First and out of range vectors:
 [-0.21475744  0.01195681] [0. 0.]
See gradient: 
 [[ -21.017763    -33.225605  ]
 [  25.473038     29.977484  ]
 [  87.87613     -12.565636  ]
 [   0.13970792  -16.665493  ]
 [  -3.736007     25.5782    ]
 [ -19.047482      5.78868   ]
 [-113.36738      12.737446  ]
 [  43.679756    -11.625079  ]]
x[0] = [-0.21473643  0.01199004]
x[N+1] = [25.47303772 29.97748375]
See gradient: 
 [[ -21.026573    -33.233444  ]
 [  25.473103     29.97312   ]
 [  88.18477     -12.55795   ]
 [   0.13928509  -16.666084  ]
 [  -3.7375975    25.580765  ]
 [ -19.048458      5.7887416 ]
 [-113.67329      12.740585  ]
 [  43.68875     -11.625734  ]]
x[0] = [-0.21469438  0.0120565 ]
x[N+1] = [25.47310257 29.97311974]
See gradient: 
 [[ -21.044193    -33.24911   ]
 [  25.473192     29.964376  ]
 [  88.8067      -12.542517  ]
 [   0.13843912  -16.667267  ]
 [  -3.7407713    25.585894  ]
 [ -19.050415      5.788865  ]
 [-114.28971      12.746799  ]
 [  

REMARK:

Based on example above, we see the flexibility of Taichi, where we could access the the gradient for individual element x.grad[i]. This is not feasible in torch.

Recall that in `torch.autograd`, it relies on $J^T \cdot \mathbf{v}$, where $\mathbf{v}$ is some indicator vector, and a for loop to construct the objective Jacobian matrix $J \in \mathbb{R}^{B \times D \times D'}$, row by row. Or for `torch.jacobian`, yeah, it's simple but returning a big tensor of many 0 $J \in \mathbb{R}^{B \times D \times B \times D'}$. 

What makes it difficult to improve on torch is that we don't have direct access to individual items. But in Taichi we have the access. There might be some kind of feasibility but I'm not sure yet!

## Kernel for Vector Value Function

We first provide an easy template for auto differentiation on vector function.

Suppose $f(x) = (x^2,x)$, we are interested in the gradient of the function:
$$
    \frac{d}{dx} f(x) |_{x=x_0}= (2x,1)|_{x=x_0}
$$

In [12]:
# We first provide an easy template for auto differentiation on vector function.

ti.init()

N = 16

x = ti.field(dtype=ti.f32, shape=N, needs_grad=True)
loss = ti.field(dtype=ti.f32, shape=(), needs_grad=True)
loss2 = ti.field(dtype=ti.f32, shape=(), needs_grad=True)

@ti.kernel
def func():
    for i in x:
       loss[None] += x[i] ** 2
       loss2[None] += x[i]

for i in range(N):
    x[i] = i

# Set the `grad` of the output variables to `1` before calling `func.grad()`.
# THIS IS VERY IMPORTANT, THIS IS SIMPLY HOW TAICHI WORKS
loss.grad[None] = 1
loss2.grad[None] = 1

func()
func.grad()
for i in range(N):
    assert x.grad[i] == i * 2 + 1


[Taichi] Starting on arch=x64


When using `kernel.grad()`, it is recommended that you always run forward kernel before backward, for example `kernel()`; `kernel.grad()` (e.g. like in the code above, we do `func()`,`func.grad()` first, who are called kernels BTW, and then calculate the gradient `x.grad`). If global fields used in the derivative calculation get mutated in the forward run, skipping kernel() breaks global data access rule #1 below and may produce incorrect gradients.

## Some Warnings on Taichi Differentiation

# Jacobian via Taichi

### No Batch Element

In [ ]:
#THIS IS NOT WORKING!!!!! SEE CORRECT EXAMPLE BELOW

import taichi as ti
import numpy as np

ti.init()#arch=ti.cpu)

n = 4  # Input dimension D
m = 3  # Output dimension D'

# y = Wx
W = ti.Matrix.field(m, n, dtype=ti.f32, shape=())  # Single W matrix
x = ti.Vector.field(n, dtype=ti.f32, shape=(), needs_grad=True)  # Input vector
y = ti.Vector.field(m, dtype=ti.f32, shape=(), needs_grad=True)  # Output vector
J = ti.Matrix.field(m, n, dtype=ti.f32, shape=())  # Jacobian storage

@ti.kernel
def compute_y():
    y[None] = W[None] @ x[None] 
# If we let the shape element be 1 dimensional, that could be the batch dimension

@ti.kernel
def compute_jacobian():
    for i in ti.static(range(m)):  # unroll the loops
        J[None][i, :] = x.grad[None]  # $\del y[i]/ \del x$ is the ith row of J

# Initialize W and x
W_np = np.random.randn(m, n).astype(np.float32)
x_np = np.random.randn(n).astype(np.float32)

W[None] = W_np 
x[None] = x_np 

# Set the `grad` of the output variables to `1` before calling `func.grad()`.
# THIS IS VERY IMPORTANT, THIS IS SIMPLY HOW TAICHI WORKS
y.grad[None] = ti.Vector([1.0] * m) 

#When using `kernel.grad()`, it is recommended that you always run forward kernel before backward, 
# for example `kernel()`; `kernel.grad()`
compute_y()
compute_y.grad() 

# All the values are already stored in x.grad, this function below is just filling in x.grad
# into the matrix we want
compute_jacobian()

print("W (Ground Truth):\n", W_np)
print("Computed Jacobian:\n", J.to_numpy()) 

#THIS IS NOT WORKING!!!!! SEE CORRECT EXAMPLE BELOW

[Taichi] Starting on arch=x64
W (Ground Truth):
 [[ 0.00785187 -0.53103006  0.9219148   0.06988239]
 [ 0.4533749   0.9868469   0.49306247 -1.3293548 ]
 [ 0.5187788   0.44597518 -2.4469428   0.4425402 ]]
Computed Jacobian:
 [[ 0.98000556  0.90179205 -1.0319655  -0.8169322 ]
 [ 0.98000556  0.90179205 -1.0319655  -0.8169322 ]
 [ 0.98000556  0.90179205 -1.0319655  -0.8169322 ]]


In [ ]:
import taichi as ti
import numpy as np

ti.init()#(arch=ti.cpu)

n = 4  # Input dimension D
m = 3  # Output dimension D'

W = ti.Matrix.field(m, n, dtype=ti.f32, shape=())
x = ti.Vector.field(n, dtype=ti.f32, shape=(), needs_grad=True)
y = ti.Vector.field(m, dtype=ti.f32, shape=(), needs_grad=True)
J = ti.Matrix.field(m, n, dtype=ti.f32, shape=())  # Jacobian storage initialized

@ti.kernel
def compute_y():
    y[None] = W[None] @ x[None]

@ti.kernel
#Parameter must be specified type in Taichi, otherwise ERROR???
def fill_jacobian_row(row_idx: ti.i32): 
    for j in ti.static(range(n)): #Unroll the for loop
        J[None][row_idx, j] = x.grad[None][j]
    #Basically 

# Randomly Initialize some values
W_np = np.random.randn(m, n).astype(np.float32)
x_np = np.random.randn(n).astype(np.float32)
W[None] = W_np
x[None] = x_np

# Fill in Jacobian row by row (Only option because that's tha "maximum" Taichi provides)
# Specificallu (dy1/dx1, dy1/dx2),,,wait 
for i in range(m):
    # Reset Gradient Wait I'm confused here
    x.grad[None] = ti.Vector(np.zeros(n, dtype=np.float32))
    y.grad[None] = ti.Vector(np.zeros(m, dtype=np.float32))
    
    # Set the `grad` of the output variables to `1` before calling `func.grad()`.
    # THIS IS VERY IMPORTANT, THIS IS SIMPLY HOW TAICHI WORKS
    y.grad[None][i] = 1.0
    
    # When using `kernel.grad()`, it is recommended that you always run forward kernel before backward, 
    # for example `kernel()`; `kernel.grad()`
    compute_y()
    compute_y.grad()
    
    # Fill in the ith row of Jacobian
    fill_jacobian_row(i)

print("Theoretical Jacobian:\n", W_np)
print("AutoDiff Result:\n", J.to_numpy())
print("Verification:", np.allclose(W_np, J.to_numpy(), atol=1e-6))

[Taichi] Starting on arch=x64
Theoretical Jacobian:
 [[-0.15131894 -2.4811885   1.327671    2.4313889 ]
 [-0.8929262   0.980089    1.3320459   1.4879698 ]
 [ 1.6262037   0.4498334  -0.50230104 -0.6511929 ]]
AutoDiff Result:
 [[-0.15131894 -2.4811885   1.327671    2.4313889 ]
 [-0.8929262   0.980089    1.3320459   1.4879698 ]
 [ 1.6262037   0.4498334  -0.50230104 -0.6511929 ]]
Verification: True


In [15]:
J[None][1,2]

1.3320459127426147